In [1]:
# # IMPORTS
# astronomy
from astropy.io import fits
# data 
import numpy as np
import csv
# my python files 
import match
import v2_AGN_DataAndPlotting as adp


In [2]:
def PrintNumber(myDict) :
    for key,val in myDict.items():
        if(isinstance(val,np.ndarray)):
            print(key,'\t',len(val)) 
        elif(isinstance(val,dict)):
            print(key,end='')
            for key1,val1 in val.items():
                print('\t',key1,'\t',len(val1))

In [3]:
# read AGN IDs
agnIDs = adp.ReadFile(adp.path_csv+'\\v2_AGNid_tricolor.csv')

# convert agnIDs into dict of integer numpy arrays 
agnIDs_dict = {
    'RED' : np.array(agnIDs[0], dtype=int),
    'GRN' : np.array(agnIDs[1], dtype=int),
    'BLU' : np.array(agnIDs[2], dtype=int)
}

# verify nmber of sources
PrintNumber(agnIDs_dict)

Cropped:  ['Row 1 = MIR exclusive AGN with id, redshift, and IRAC (n=422). Row 2 = MIR-X-ray inclusive AGN with id, redshift, and IRAC (n=671). Row 3 = X-ray exclusive AGN with id, redshift, and IRAC (n=1638). ']
RED 	 422
GRN 	 671
BLU 	 1638


In [4]:
# get redshifts
zDict = adp.ReadRedshifts(file=adp.path_cat+'\COSMOS_z_matches.csv')

Cropped:  ['ID', 'z']
Number of redshifts: 487252


In [5]:
agnInfo = {}
for key,val in agnIDs_dict.items():
    # match redshift to AGN
    key_zid, key_idz = match.match(zDict['ID'], val)
    agnInfo[key] = {
        'ID' : val[key_idz],
        'z'  : zDict['z'][key_zid]
    }
# verify number of sources
PrintNumber(agnInfo)

RED	 ID 	 422
	 z 	 422
GRN	 ID 	 671
	 z 	 671
BLU	 ID 	 1638
	 z 	 1638


In [6]:
# open COSMOS 2020 catalog file and get data 
with fits.open(adp.path_cat+'\\COSMOS2020_CLASSIC_R1_v2.0_master.fits') as hdul20:
    data20 = hdul20[1].data
    cols20 = hdul20[1].columns

In [7]:
# # open COSMOS 2016 catalog file and get data 
# with fits.open(adp.path_cat+'\\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul16:
#     data16 = hdul16[1].data
#     cols16 = hdul16[1].columns

In [8]:
# # get data arrays from columns
# id_data16 = np.array(data16['id_k_uv'])
# print('.', end='')
# L0510_data16 = np.array(data16['Lx_0510']) # x-ray luminosity in the 0.5-10 KeV band
# print('.', end='')
# k0510_data16 = np.array(data16['abs_corr_0510'])  # absorbtion correction coefficient in the 0.5-10 KeV band
# print('.', end='')

In [9]:
# read file 
dataNew = adp.UnpackTable(adp.path_csv+'\\New_Lx_cat.csv')

# fix types
dataNew['ID']   = np.array(dataNew['ID'], dtype=int)
dataNew['z']    = np.array(dataNew['z'],  dtype=float)
dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale

# replace C-COSMOS 2016 for X-ray selection 
id_data16 = dataNew['ID']
L0510_data16 = dataNew['Lx_05_10']

C:\Users\polar\AppData\Local\Temp\ipykernel_20276\2962619769.py:7: RuntimeWarning: divide by zero encountered in log10
  dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_20276\2962619769.py:7: RuntimeWarning: invalid value encountered in log10
  dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_20276\2962619769.py:8: RuntimeWarning: divide by zero encountered in log10
  dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_20276\2962619769.py:8: RuntimeWarning: invalid value encountered in log10
  dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale


In [10]:
# add Lx to dict 
for key,val in agnInfo.items() : 
    # match ids
    idagn, id16 = match.match(val['ID'], id_data16)
    # if all agns have a match
    if(len(id16) == len(val['ID'])) : 
        # fast match
        # Lx = adp.IntrinsicLuminosity(
        #         L0510_data16[id16],
        #         k0510_data16[id16]
        #     )
        Lx = L0510_data16[id16]
    # some agns do not have Lx data
    else : 
        idsWithLx = id_data16[id16]
        Lx = []
        TEST = []
        # for each agn id
        for i in range(len(val['ID'])) : 
            thisID = val['ID'][i]
            # check if the agn has Lx data
            if( thisID in idsWithLx) : 
                # get Lx
                i = np.where(id_data16 == thisID)[0][0]
                TEST.append(id_data16[i])
                # Lx.append(
                #     adp.IntrinsicLuminosity(
                #         L0510_data16[[i]],
                #         k0510_data16[[i]]
                #     )[0]
                # )
                Lx.append( (L0510_data16[[i]])[0] )
            # agn has no Lx data
            else : 
                Lx.append(-99)
    # add to info
    val['Lx'] = Lx

# verify number of sources
PrintNumber(agnInfo)

RED	 ID 	 422
	 z 	 422
	 Lx 	 422
GRN	 ID 	 671
	 z 	 671
	 Lx 	 671
BLU	 ID 	 1638
	 z 	 1638
	 Lx 	 1638


In [11]:
with open(adp.path_csv+'\\v2_AGN_tricolor_data.csv', mode='w',newline='') as csv_file:
    # initialize csv
    writer = csv.writer(csv_file)
    # write column names 
    writer.writerow(['ID', 'Group', 'z', 'Lx'])
    # for each agn group
    for key,val in agnInfo.items() : 
        # for each agn source
        for i in range(len(val['ID'])) :
            # write data 
            writer.writerow([ val['ID'][i], key,val['z'][i],val['Lx'][i]])
    # confirm 
    print('List saved: v2_AGN_tricolor_data.csv')

List saved: v2_AGN_tricolor_data.csv
